In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
df = pd.read_csv('train.csv')
df.shape

(891, 12)

In [5]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [7]:
df['Pclass'].unique()

array([3, 1, 2])

In [15]:
columns_to_ignore = ('PassengerId', 'Name', 'Ticket')

for column in df.columns:
    if column not in columns_to_ignore:
        print(column, df[column].unique())

Survived [0 1]
Pclass [3 1 2]
Sex ['male' 'female']
Age [ 22.    38.    26.    35.      nan  54.     2.    27.    14.     4.    58.
  20.    39.    55.    31.    34.    15.    28.     8.    19.    40.    66.
  42.    21.    18.     3.     7.    49.    29.    65.    28.5    5.    11.
  45.    17.    32.    16.    25.     0.83  30.    33.    23.    24.    46.
  59.    71.    37.    47.    14.5   70.5   32.5   12.     9.    36.5   51.
  55.5   40.5   44.     1.    61.    56.    50.    36.    45.5   20.5   62.
  41.    52.    63.    23.5    0.92  43.    60.    10.    64.    13.    48.
   0.75  53.    57.    80.    70.    24.5    6.     0.67  30.5    0.42
  34.5   74.  ]
SibSp [1 0 3 4 2 5 8]
Parch [0 1 2 5 3 4 6]
Fare [   7.25     71.2833    7.925    53.1       8.05      8.4583   51.8625
   21.075    11.1333   30.0708   16.7      26.55     31.275     7.8542
   16.       29.125    13.       18.        7.225    26.        8.0292
   35.5      31.3875  263.        7.8792    7.8958   27.7208  1

In [37]:
VALID = df.shape[0] // 9 # 1/9th will be "dev set" data

In [40]:
all_df = df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
test_df = all_df[:-VALID]
dev_df = all_df[:VALID]

print(test_df.shape)
print(dev_df.shape)

(792, 9)
(99, 9)


In [42]:
all_df['Age'].unique()

array([ 22.  ,  38.  ,  26.  ,  35.  ,    nan,  54.  ,   2.  ,  27.  ,
        14.  ,   4.  ,  58.  ,  20.  ,  39.  ,  55.  ,  31.  ,  34.  ,
        15.  ,  28.  ,   8.  ,  19.  ,  40.  ,  66.  ,  42.  ,  21.  ,
        18.  ,   3.  ,   7.  ,  49.  ,  29.  ,  65.  ,  28.5 ,   5.  ,
        11.  ,  45.  ,  17.  ,  32.  ,  16.  ,  25.  ,   0.83,  30.  ,
        33.  ,  23.  ,  24.  ,  46.  ,  59.  ,  71.  ,  37.  ,  47.  ,
        14.5 ,  70.5 ,  32.5 ,  12.  ,   9.  ,  36.5 ,  51.  ,  55.5 ,
        40.5 ,  44.  ,   1.  ,  61.  ,  56.  ,  50.  ,  36.  ,  45.5 ,
        20.5 ,  62.  ,  41.  ,  52.  ,  63.  ,  23.5 ,   0.92,  43.  ,
        60.  ,  10.  ,  64.  ,  13.  ,  48.  ,   0.75,  53.  ,  57.  ,
        80.  ,  70.  ,  24.5 ,   6.  ,   0.67,  30.5 ,   0.42,  34.5 ,  74.  ])

In [43]:
# 1st attempt, by "age"

# Specify that all features have real-value data
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                        hidden_units=[10, 20, 10],
                                        n_classes=3,
                                        model_dir="/tmp/titanic")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/titanic', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1145bea58>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [73]:
test_df.loc[:, 'Age'] = test_df['Age'].fillna(0)

age_data = StandardScaler().fit_transform(np.float32(test_df['Age'].values[:, None]))

/Users/aaron/Documents/ml/kaggle/digit-recognizer/venv/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [78]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(test_df['Age'].values)},
    y=np.array(test_df['Survived'].fillna(0).values),
    num_epochs=None,
    shuffle=True)

In [79]:
# Train model.
classifier.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/titanic/model.ckpt-3
INFO:tensorflow:Saving checkpoints for 4 into /tmp/titanic/model.ckpt.
INFO:tensorflow:loss = 99.089, step = 4
INFO:tensorflow:global_step/sec: 540.173
INFO:tensorflow:loss = 81.9191, step = 104 (0.186 sec)
INFO:tensorflow:global_step/sec: 619.159
INFO:tensorflow:loss = 82.2905, step = 204 (0.161 sec)
INFO:tensorflow:global_step/sec: 636.853
INFO:tensorflow:loss = 85.6751, step = 304 (0.157 sec)
INFO:tensorflow:global_step/sec: 593.927
INFO:tensorflow:loss = 78.4316, step = 404 (0.169 sec)
INFO:tensorflow:global_step/sec: 638.316
INFO:tensorflow:loss = 85.1989, step = 504 (0.156 sec)
INFO:tensorflow:global_step/sec: 599.068
INFO:tensorflow:loss = 83.6208, step = 604 (0.167 sec)
INFO:tensorflow:global_step/sec: 619.352
INFO:tensorflow:loss = 91.2771, step = 704 (0.162 sec)
INFO:tensorflow:global_step/sec: 631.97
INFO:tensorflow:loss = 80.7976, step = 804 (0.158 sec)
INFO:tenso

In [81]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(dev_df['Age'].values)},
    y=np.array(dev_df['Survived'].fillna(0).values),
    num_epochs=1,
    shuffle=False)

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

INFO:tensorflow:Starting evaluation at 2018-01-13-22:48:36
INFO:tensorflow:Restoring parameters from /tmp/titanic/model.ckpt-2003
INFO:tensorflow:Finished evaluation at 2018-01-13-22:48:37
INFO:tensorflow:Saving dict for global step 2003: accuracy = 0.585859, accuracy_baseline = 0.585859, auc = 0.485702, auc_precision_recall = 0.427263, average_loss = 0.69295, global_step = 2003, label/mean = 0.414141, loss = 68.6021, prediction/mean = 0.37602

Test Accuracy: 0.585859

